In [3]:
import pandas as pd
import numpy as np
import networkx as nx

In [69]:
FILENAME_STRUCTURE = 'scraper-data/postgraduateforum-2104.csv'
FILENAME_SIM = 'watt-03032020-adj.npy'

In [80]:
def remove_protocol(url):
    assert url.startswith('http'), f'Formatting error: URL "{url}" not valid.'
    
    if url.startswith('http://'):
        return url[7:]
    else:
        return url[8:]

## Load data

In [81]:
structure = pd.read_csv(FILENAME_STRUCTURE, usecols=['url', 'connected_to'])
structure['url'] = structure['url'].apply(remove_protocol)
structure.head()

,url,connected_to
0,www.postgraduateforum.com/,"https://www.postgraduateforum.com/,https://www..."
1,www.postgraduateforum.com/tricky-profile,"https://www.postgraduateforum.com/,https://www..."
2,www.postgraduateforum.com/lara-profile,"https://www.postgraduateforum.com/,https://www..."
3,www.postgraduateforum.com/sneaks-profile,"https://www.postgraduateforum.com/,https://www..."
4,www.postgraduateforum.com/docinsanity-blog/com...,"https://www.postgraduateforum.com/,https://www..."


## Create the graph of the structure

### Check if there are duplicates

In [82]:
unique_urls, count_duplicates = np.unique(structure['url'].values, return_counts=True)
duplicate_urls = unique_urls[count_duplicates > 1]
print('Duplicate URLs:', duplicate_urls)

Duplicate URLs: ['www.postgraduateforum.com/']


In [83]:
structure_graph = nx.Graph()
structure_graph.add_nodes_from(structure['url'].values)

In [87]:
for row in structure['connected_to'].dropna():
    urls = map(remove_protocol, row.split(','))
    for url in urls:
        # Don't consider connections which are not pages themselves
        if url in structure_graph:
            structure_graph.add_edge(row, url)

## Load the content adj matrix

In [57]:
content_adj = np.load(FILENAME_SIM)